In [63]:
import pandas as pd
import sqlite3

In [64]:
# Chargement des données
data = pd.read_excel(r'Test hist.xlsx', sheet_name="Data")
fact = pd.read_excel(r'Test hist.xlsx', sheet_name="Fact")

In [65]:
#Création d'une base de donnée in memory
conn = sqlite3.connect(':memory:')

In [66]:
# Ajout des Dataframe dans la base de données
data.to_sql('data', conn, index=False)
fact.to_sql('fact', conn, index=False)

In [73]:
# Création d'une requête sql
req = "select data.Fun_key, Beg, End, Date, Tech_key from Fact left join Data on data.Fun_key=fact.Fun_key and Date between Beg and End"

In [71]:
# Exécution de la requête et stokage du résultat dans un DataFrame
df = pd.read_sql_query(req, conn)

In [72]:
print(df)

   Fun_key                  Beg                 Date  Tech_key
0        1  2011-04-24 00:00:00  2016-04-13 00:00:00         2
1        2  2011-04-24 00:00:00  2016-03-01 00:00:00         4


In [202]:
def date_merge(source_table, target_table, source_functional_key, target_functional_key, source_date_key,  target_column_to_get, target_date_columns_name=[]):
    # Fonction servant à récupérer une colonne à l'aide d'une clé fonctionnelle et d'un date between. Cette fonction complète une lacune de la bibliothèque Pandas
    # Il est nécessaire d'importer la bibliothèque sqlite3
    
    # source table => Dataframe source qui va être alimenté
    # target_table => Dataframe servant à alimenter le dataframe source
    # source_functional_key => la clé fonctionnelle servant à créer la jointure
    # target_functionnal_key=> la clé fonctionnelle cible servant à créer la jointure en dehors des dates
    # source_date_key => la date provenant du DataFrame source servant à la condition in beetween
    # target_column_to_get => la colonne à récupérer
    # target_date_columns_name => le nom des colonnes à récupérer sans espace, remplacer les espace par "_", ex: toto titi, toto_titi / Ces colonnes doivent être des pandas datetimes
    
    conn = sqlite3.connect(':memory:')
    source_table.to_sql('source', conn, index=False)
    target_table.to_sql('target', conn, index=False)
    
    #Création de la requète
    req = "PRAGMA table_info(source);"
    col_source_prep = pd.read_sql_query(req, conn)
    col_source_prep = list(col_source_prep["name"])
    col_source = []
    
    for val in col_source_prep :
        col_source.append('source.' + str(val)) 
        
    col_source = str(col_source)
    col_source = col_source.replace("[", "")
    col_source = col_source.replace("]", "")
    col_source = col_source.replace("'","")

    req = f'Select {col_source}, {target_column_to_get} from source left join target on source.{source_functional_key}=target.{target_functional_key} and source.{source_date_key} between {target_date_columns_name[0]} and {target_date_columns_name[1]}'
    
    df = pd.read_sql_query(req, conn)
    
    return df
    
    

In [203]:
date_merge(source_table=fact, target_table=data, source_functional_key='Fun_key',target_functional_key='Fun_key',
           source_date_key='Date', target_date_columns_name= ["Beg","End"], target_column_to_get="Tech_key")

,Fun_key,Date,Tech_key
0,1,2016-04-13 00:00:00,2
1,2,2016-03-01 00:00:00,4
